In [48]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
import os
os.environ['WANDB_DISABLED'] = 'true'
HOME = os.getcwd()
print(HOME)
!pip install ultralytics==8.2.103 -q
from ultralytics import YOLO
import numpy as np
from IPython.display import display, Image

from google.colab import drive
drive.mount('/content/drive')

# Funcție pentru aplicarea Watershed ca layer peste imaginea originală
def apply_watershed_overlay(image, bbox):
    x_center, y_center, width, height = bbox
    x1 = int(x_center - width / 2)
    y1 = int(y_center - height / 2)
    x2 = int(x_center + width / 2)
    y2 = int(y_center + height / 2)



    x1, y1 = max(0, x1), max(0, y1)
    x2, y2 = min(image.shape[1], x2), min(image.shape[0], y2)

    roi = image[y1:y2, x1:x2].copy()
    print(f"ROI shape: {roi.shape}")  # Verifică dimensiunile ROI-ului
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    blurred = cv2.medianBlur(gray, 9)
    edges = cv2.Canny(blurred, 150, 190)

    # Markerii pentru Watershed
    ret, thresh = cv2.threshold(edges, 190, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel = np.ones((3, 3), np.uint8)
    sure_bg = cv2.dilate(thresh, kernel, iterations=2)
    dist_transform = cv2.distanceTransform(thresh, cv2.DIST_L2, 5)
    ret, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)
    sure_bg = sure_bg.astype(np.uint8)
    sure_fg = sure_fg.astype(np.uint8)
    unknown = cv2.subtract(sure_bg, sure_fg)

    # Etichetarea markerilor
    sure_fg = np.uint8(sure_fg)
    ret, markers = cv2.connectedComponents(sure_fg)
    markers = markers + 1

    # Aici este modificarea
    markers = markers.astype(np.int32) # Asigurați-vă că markerii sunt de tipul corect

    markers[unknown == 255] = 0


    # Aplicarea Watershed
    cv2.watershed(roi, markers)

    # Creăm un layer cu contururile detectate
    overlay = np.zeros_like(roi)
    overlay[markers == -1] = [255, 0, 255]  # Contururi roșii

    # Suprapunem layer-ul peste imaginea originală
    blended = cv2.addWeighted(roi, 1, overlay, 1, 0)  # 0.3 pentru transparență

    # Înlocuim doar ROI-ul procesat în imaginea originală
    result_image = image.copy()
    result_image[y1:y2, x1:x2] = blended
    return result_image


model = YOLO('/content/drive/MyDrive/PROIECT_PI/runs/detect/train2/weights/best.pt')
#results = model.predict(source=dataset_path1, save=True,imgsz=640)
#result_images = glob.glob('/content/drive/MyDrive/PROIECT_PI/runs/detect/train2/Predictie/test_predictions/*.jpg')
dataset_path1 = '/content/drive/MyDrive/YOLO_dataset/train2/images'
#model.train(data='/content/drive/MyDrive/YOLO_dataset/data2.yaml',epochs=50,batch=16,imgsz=640)
#metrics = model.val(data='/content/drive/MyDrive/YOLO_dataset/data2.yaml', split='test')
import glob

#Acum vom face preprocesarea si vom testa daca se va schimba ceva in detectie
import cv2
input_folder='/content/drive/MyDrive/YOLO_dataset/test2/images'
output_folder='/content/drive/MyDrive/PROIECT_PI/runs/detect/train2/Preprocesarea_Imaginilor'
results = model.predict(source=dataset_path1, save=True,imgsz=640)
result_images = glob.glob('/content/drive/MyDrive/PROIECT_PI/runs/detect/train2/Predictie/test_predictions/*.jpg')
for filename in os.listdir(input_folder):
    if filename.endswith(('.jpg', '.png', '.jpeg')):
        img_path = os.path.join(input_folder, filename)
        img = cv2.imread(img_path,cv2.IMREAD_COLOR)
        for result in results:
          if result.boxes:
            for box1 in result.boxes:
              x1,y1,x2,y2 = box1.xyxy[0]
              x1, y1 = max(0, x1), max(0, y1)
              x2, y2 = min(img.shape[1], x2), min(img.shape[0], y2)

            # Continuă procesarea doar dacă coordonatele sunt valide
              x_center = (x1 + x2) / 2
              y_center = (y1 + y2) / 2
              width = x2 - x1
              height = y2 - y1
              bbox_xywh = (x_center, y_center, width, height)

                # Aplică Watershed
              img_with_overlay = apply_watershed_overlay(img, bbox_xywh)

              output_path = os.path.join(output_folder, filename)
              cv2.imwrite(output_path, img_with_overlay)












Streaming output truncated to the last 5000 lines.
ROI shape: (185, 459, 3)
ROI shape: (190, 446, 3)
ROI shape: (228, 497, 3)
ROI shape: (234, 456, 3)
ROI shape: (20, 165, 3)
ROI shape: (236, 457, 3)
ROI shape: (238, 495, 3)
ROI shape: (245, 514, 3)
ROI shape: (637, 629, 3)
ROI shape: (637, 633, 3)
ROI shape: (192, 633, 3)
ROI shape: (199, 639, 3)
ROI shape: (191, 630, 3)
ROI shape: (141, 482, 3)
ROI shape: (254, 591, 3)
ROI shape: (256, 569, 3)
ROI shape: (328, 640, 3)
ROI shape: (76, 474, 3)
ROI shape: (332, 640, 3)
ROI shape: (82, 456, 3)
ROI shape: (328, 640, 3)
ROI shape: (76, 493, 3)
ROI shape: (335, 638, 3)
ROI shape: (80, 451, 3)
ROI shape: (593, 640, 3)
ROI shape: (611, 640, 3)
ROI shape: (590, 639, 3)
ROI shape: (612, 640, 3)
ROI shape: (626, 640, 3)
ROI shape: (617, 639, 3)
ROI shape: (315, 356, 3)
ROI shape: (43, 212, 3)
ROI shape: (317, 363, 3)
ROI shape: (37, 221, 3)
ROI shape: (49, 245, 3)
ROI shape: (312, 352, 3)
ROI shape: (41, 212, 3)
ROI shape: (312, 357, 3)
ROI shap

KeyboardInterrupt: 